In [1]:
import polars as pl
import polars.selectors as cs
from glob import glob
import re

In [3]:
(water_quality := (
    pl.scan_csv(
        'data/MinneMUDAC_raw_files/water_quality_and_parcel_summaries_2004_to_2015.csv',
        has_header=True,
        separator=',',
        infer_schema_length=10000,
        schema_overrides={'pH': pl.Float64}
    )
    .limit(100)
    .collect(
    )
)
)

,Monit_MAP_CODE1,Year,LAKE_NAME,Mean_Secchi_Depth_Result,Mean_Phosporus_Result,Mean_EMV_Total,STD_EMV_Total,Mean_Sale_Value,STD_Sale_Value,Mean_Total_Tax,STD_Total_Tax,Mean_Garage_Size,STD_Garage_Size,Mean_Fin_SQ_FT,STD_Fin_SQ_FT,Percentage_Yes_Basement,Percentage_Yes_Garage,Percentage_Yes_Tax_Exempt,Percentage_Air_Cooling,Percentage_AC_Cooling,Percentage_Central_Cooling,Percentage_Other_Cooling,Percentage_No_Cooling,Percentage_Air_Heating,Percentage_Space_Heating,Percentage_Water_Heating,Percentage_Electric_Heating,Percentage_Other_Heating,Percentage_No_Heating
i64,str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,"""02000500-01""",2014,"""George Watch Lake""",0.716667,0.108778,215637.592745,459023.922281,122485.615829,183414.899771,3313.685903,17837.814516,null,null,1849.473207,8220.487298,70.486397,0.0,7.914262,0.0,0.0,0.0,2.308326,97.691674,0.0,0.0,0.0,0.0,85.325639,14.674361
1,"""02000500-01""",2013,"""George Watch Lake""",0.365,0.3105,196764.633141,454192.377508,120601.723001,190129.461399,3319.081616,18199.952917,null,null,1841.112119,8221.467234,69.991756,0.0,7.914262,0.824402,0.0,0.0,1.154163,0.0,79.060181,0.824402,1.154163,2.802968,1.154163,0.0
2,"""02000500-01""",2012,"""George Watch Lake""",0.359,0.2649,200414.333057,480355.968668,118315.128418,190091.029827,3460.064623,19346.923479,null,null,1811.576636,8223.512958,70.173985,0.0,8.119304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"""02000500-01""",2011,"""George Watch Lake""",0.973333,0.119417,216297.932175,536941.029067,111218.812242,190312.451914,3459.313482,19320.28833,null,null,1780.282051,8215.092878,76.840364,0.0,8.271299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"""02000500-01""",2010,"""George Watch Lake""",0.493333,0.173,222789.430223,559656.083477,110765.119736,190448.272924,3384.645747,18646.626107,null,null,1760.876135,8210.519477,75.887696,0.0,8.092486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
95,"""19002200-01""",2007,"""Long Lake""",1.733766,0.102672,302888.441533,471157.778953,141962.587555,127724.729722,3097.440955,3068.087373,null,null,2275.401657,2624.866538,0.0,0.0,2.273165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,"""19002200-01""",2006,"""Long Lake""",1.459746,0.111475,298946.734733,459972.909725,137663.425544,126084.446256,3015.460412,2593.59527,null,null,2266.360239,2642.997239,0.0,0.0,2.273165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,"""19002200-01""",2005,"""Long Lake""",1.463409,0.082091,276705.113856,416702.275065,132619.693941,122315.844073,2491.457545,1530.016612,null,null,2247.141258,2611.387698,0.0,0.0,2.315708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Task 1
We want to extract 30% of the lakes to be used as a validation set, with the remaining lakes separated into a train/test split (again use 70%/30% split).In summary we should have a random split with
1. 30% of the lakes marked as "Validation",
2. 50% of the lakes (~70% of 70%) marked as "Training", and
3. 20% of the lakes (~30% of 70%) marked as "Test".

In [30]:
unique_lakes = water_quality['LAKE_NAME'].unique().to_list()
print(f"Number of unique lakes: {len(unique_lakes)}")
print("First 5 unique lake names:")
print(unique_lakes[:5])

Number of unique lakes: 10
First 5 unique lake names:
['Alimagnet Lake', 'Riley Lake', 'George Watch Lake', 'Long Lake', 'Bavaria Lake']


## Split Lakes into Validation, Training, and Test Sets

### Subtask:
Randomly assign each unique lake to one of three groups: 'Validation' (30% of lakes), 'Training' (50% of lakes), or 'Test' (20% of lakes). This ensures that lakes are not duplicated across different sets.


In [41]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

shuffled_lakes = np.array(unique_lakes)
np.random.shuffle(shuffled_lakes)

# Split into Test (20%) and Training/Validation (80%)
train_val_lakes, test_lakes = train_test_split(shuffled_lakes, test_size=0.2, random_state=42)

# Split Training/Validation (80%) into Training (50% of total) and Validation (30% of total)
# Since train_val_lakes is 80% of the total, 50% of total is 50/80 = 0.625 of train_val_lakes
# And 30% of total is 30/80 = 0.375 of train_val_lakes
train_lakes, val_lakes = train_test_split(train_val_lakes, test_size=0.375, random_state=42) # 0.375 of 80% = 30% of total

print(f"Number of unique lakes: {len(unique_lakes)}")
print(f"Number of lakes in Training set: {len(train_lakes)}")
print(f"Number of lakes in Validation set: {len(val_lakes)}")
print(f"Number of lakes in Test set: {len(test_lakes)}")

print(f"\nTraining set proportion: {len(train_lakes) / len(unique_lakes):.2f}")
print(f"Validation set proportion: {len(val_lakes) / len(unique_lakes):.2f}")
print(f"Test set proportion: {len(test_lakes) / len(unique_lakes):.2f}")

Number of unique lakes: 10
Number of lakes in Training set: 5
Number of lakes in Validation set: 3
Number of lakes in Test set: 2

Training set proportion: 0.50
Validation set proportion: 0.30
Test set proportion: 0.20


In [31]:
lake_to_split_group = {}
for lake in train_lakes:
    lake_to_split_group[lake] = 'Training'
for lake in val_lakes:
    lake_to_split_group[lake] = 'Validation'
for lake in test_lakes:
    lake_to_split_group[lake] = 'Test'


split_group_df = pl.DataFrame({
    'LAKE_NAME': list(lake_to_split_group.keys()),
    'Split_Group': list(lake_to_split_group.values())
})


water_quality = water_quality.join(split_group_df, on='LAKE_NAME', how='left')

print(water_quality.head())

shape: (5, 32)
┌─────┬─────────────┬──────┬─────────────┬───┬─────────────┬─────────────┬────────────┬────────────┐
│     ┆ Monit_MAP_C ┆ Year ┆ LAKE_NAME   ┆ … ┆ Percentage_ ┆ Percentage_ ┆ Split_Grou ┆ Split_Grou │
│ --- ┆ ODE1        ┆ ---  ┆ ---         ┆   ┆ Other_Heati ┆ No_Heating  ┆ p          ┆ p_right    │
│ i64 ┆ ---         ┆ i64  ┆ str         ┆   ┆ ng          ┆ ---         ┆ ---        ┆ ---        │
│     ┆ str         ┆      ┆             ┆   ┆ ---         ┆ f64         ┆ str        ┆ str        │
│     ┆             ┆      ┆             ┆   ┆ f64         ┆             ┆            ┆            │
╞═════╪═════════════╪══════╪═════════════╪═══╪═════════════╪═════════════╪════════════╪════════════╡
│ 0   ┆ 02000500-01 ┆ 2014 ┆ George      ┆ … ┆ 85.325639   ┆ 14.674361   ┆ Test       ┆ Test       │
│     ┆             ┆      ┆ Watch Lake  ┆   ┆             ┆             ┆            ┆            │
│ 1   ┆ 02000500-01 ┆ 2013 ┆ George      ┆ … ┆ 1.154163    ┆ 0.0         ┆ T

In [42]:
lake_to_split_group = {}
for lake in train_lakes:
    lake_to_split_group[lake] = 'Training'
for lake in val_lakes:
    lake_to_split_group[lake] = 'Validation'
for lake in test_lakes:
    lake_to_split_group[lake] = 'Test'

split_group_df = pl.DataFrame({
    'LAKE_NAME': list(lake_to_split_group.keys()),
    'Split_Group': list(lake_to_split_group.values())
})

water_quality = water_quality.join(split_group_df, on='LAKE_NAME', how='left')

print(water_quality.head())

shape: (5, 32)
┌─────┬─────────────┬──────┬─────────────┬───┬─────────────┬─────────────┬────────────┬────────────┐
│     ┆ Monit_MAP_C ┆ Year ┆ LAKE_NAME   ┆ … ┆ Percentage_ ┆ Percentage_ ┆ Split_Grou ┆ Split_Grou │
│ --- ┆ ODE1        ┆ ---  ┆ ---         ┆   ┆ Other_Heati ┆ No_Heating  ┆ p          ┆ p_right    │
│ i64 ┆ ---         ┆ i64  ┆ str         ┆   ┆ ng          ┆ ---         ┆ ---        ┆ ---        │
│     ┆ str         ┆      ┆             ┆   ┆ ---         ┆ f64         ┆ str        ┆ str        │
│     ┆             ┆      ┆             ┆   ┆ f64         ┆             ┆            ┆            │
╞═════╪═════════════╪══════╪═════════════╪═══╪═════════════╪═════════════╪════════════╪════════════╡
│ 0   ┆ 02000500-01 ┆ 2014 ┆ George      ┆ … ┆ 85.325639   ┆ 14.674361   ┆ Test       ┆ Test       │
│     ┆             ┆      ┆ Watch Lake  ┆   ┆             ┆             ┆            ┆            │
│ 1   ┆ 02000500-01 ┆ 2013 ┆ George      ┆ … ┆ 1.154163    ┆ 0.0         ┆ T

In [34]:
# water_quality = water_quality.drop('Split_Group_right') # This line caused the error, so it's removed.

# Verify and display the distribution of lakes across the three sets
split_distribution = water_quality['Split_Group'].value_counts(normalize=True)
# Polars' value_counts(normalize=True) returns a DataFrame with 'proportion' column
split_distribution = split_distribution.with_columns(
    (pl.col('proportion') * 100).round(2).alias('percentage')
).select(pl.col('Split_Group'), pl.col('percentage'))

print("Distribution of samples across split groups:")
print(split_distribution)

Distribution of samples across split groups:
shape: (3, 2)
┌─────────────┬────────────┐
│ Split_Group ┆ percentage │
│ ---         ┆ ---        │
│ str         ┆ f64        │
╞═════════════╪════════════╡
│ Test        ┆ 22.0       │
│ Validation  ┆ 33.0       │
│ Training    ┆ 45.0       │
└─────────────┴────────────┘


In [35]:
split_distribution = (
    water_quality['Split_Group']
    .value_counts(normalize=True)
    .with_columns(
        (pl.col('proportion') * 100).round(2).alias('percentage')
    )
    .select(pl.col('Split_Group'), pl.col('percentage'))
)
print("Distribution of samples across split groups:")
print(split_distribution)

Distribution of samples across split groups:
shape: (3, 2)
┌─────────────┬────────────┐
│ Split_Group ┆ percentage │
│ ---         ┆ ---        │
│ str         ┆ f64        │
╞═════════════╪════════════╡
│ Validation  ┆ 33.0       │
│ Test        ┆ 22.0       │
│ Training    ┆ 45.0       │
└─────────────┴────────────┘


In [40]:
import polars as pl

# Assuming water_quality is a Polars DataFrame
split_distribution = (
    water_quality['Split_Group']                  # Select the column as a Series
    .value_counts(sort=True, normalize=True)      # Get counts + proportion (normalized fraction)
    .with_columns(
        (pl.col("proportion") * 100)               # Use 'proportion' column
        .round(2)
        .alias("percentage")
    )
    .select("Split_Group", "percentage")      # Keep only what you want
)

print("Distribution of samples across split groups:")
print(split_distribution)

Distribution of samples across split groups:
shape: (3, 2)
┌─────────────┬────────────┐
│ Split_Group ┆ percentage │
│ ---         ┆ ---        │
│ str         ┆ f64        │
╞═════════════╪════════════╡
│ Training    ┆ 45.0       │
│ Validation  ┆ 33.0       │
│ Test        ┆ 22.0       │
└─────────────┴────────────┘


In [16]:
split_distribution = (
    water_quality['Split_Group']
    .value_counts(normalize=True)
    .with_columns(
        (pl.col('proportion') * 100).round(2).alias('percentage')
    )
    .select(pl.col('Split_Group'), pl.col('percentage'))
)
print("Distribution of samples across split groups:")
print(split_distribution)

Distribution of samples across split groups:
shape: (3, 2)
┌─────────────┬────────────┐
│ Split_Group ┆ percentage │
│ ---         ┆ ---        │
│ str         ┆ f64        │
╞═════════════╪════════════╡
│ Training    ┆ 45.0       │
│ Test        ┆ 22.0       │
│ Validation  ┆ 33.0       │
└─────────────┴────────────┘


## Summary:

### Q&A
Yes, the split was successfully created. 10 unique lakes were split into 'Training' (5 lakes, 50%), 'Validation' (3 lakes, 30%), and 'Test' (2 lakes, 20%) sets. A `Split_Group` column was added to the `water_quality` DataFrame. The distribution of records (samples) in the `water_quality` DataFrame across these groups is 45.0% for Training, 22.0% for Test, and 33.0% for Validation.

### Data Analysis Key Findings
*   A total of 10 unique lakes were identified in the `water_quality` DataFrame.
*   These unique lakes were successfully split into:
    *   Training set: 5 lakes, representing 50% of unique lakes.
    *   Validation set: 3 lakes, representing 30% of unique lakes.
    *   Test set: 2 lakes, representing 20% of unique lakes.
*   A new `Split_Group` column was successfully added to the `water_quality` DataFrame, assigning each record to its respective 'Training', 'Validation', or 'Test' group based on its `LAKE_NAME`.
*   The distribution of *samples* (records) within the `water_quality` DataFrame across the split groups is:
    *   Training: 45.0%
    *   Test: 22.0%
    *   Validation: 33.0%
    *   These percentages differ slightly from the initial lake split percentages (50%, 20%, 30%) because the number of data points per lake in the `water_quality` DataFrame varies.

### Insights or Next Steps
*   The established lake-based split ensures that all data points for a given lake are consistently in one set, which is crucial for preventing data leakage when modeling, especially if lake-specific features or patterns are significant.
*   While the sample distribution slightly deviates from the target lake distribution, this is expected given the varying number of records per lake. Further analysis could explore whether this sample distribution bias significantly impacts model training or evaluation, or if a different splitting strategy (e.g., balancing records per lake) is required for specific modeling objectives.


# Task 2

For each target,

1. Perform a grid search for both CART and Random Forests using the training lakes,

2. Compare the best model of each type on the test lakes to determine the best overall model.

3. Refit the best model on the 70% of the lakes not in the validation set (training + test).

4. Use the validation set to estimate the performance of this model.

In [18]:
print(water_quality.columns)


['', 'Monit_MAP_CODE1', 'Year', 'LAKE_NAME', 'Mean_Secchi_Depth_Result', 'Mean_Phosporus_Result', 'Mean_EMV_Total', 'STD_EMV_Total', 'Mean_Sale_Value', 'STD_Sale_Value', 'Mean_Total_Tax', 'STD_Total_Tax', 'Mean_Garage_Size', 'STD_Garage_Size', 'Mean_Fin_SQ_FT', 'STD_Fin_SQ_FT', 'Percentage_Yes_Basement', 'Percentage_Yes_Garage', 'Percentage_Yes_Tax_Exempt', 'Percentage_Air_Cooling', 'Percentage_AC_Cooling', 'Percentage_Central_Cooling', 'Percentage_Other_Cooling', 'Percentage_No_Cooling', 'Percentage_Air_Heating', 'Percentage_Space_Heating', 'Percentage_Water_Heating', 'Percentage_Electric_Heating', 'Percentage_Other_Heating', 'Percentage_No_Heating', 'Split_Group']


In [19]:
target_variables = [
    'Mean_Secchi_Depth_Result',
    'Mean_Phosporus_Result'
]

print(f"Selected target variables: {target_variables}")

Selected target variables: ['Mean_Secchi_Depth_Result', 'Mean_Phosporus_Result']


In [20]:
train_df = water_quality.filter(pl.col('Split_Group') == 'Training')
val_df = water_quality.filter(pl.col('Split_Group') == 'Validation')
test_df = water_quality.filter(pl.col('Split_Group') == 'Test')

print(f"Shape of training DataFrame: {train_df.shape}")
print(f"Shape of validation DataFrame: {val_df.shape}")
print(f"Shape of test DataFrame: {test_df.shape}")

Shape of training DataFrame: (45, 31)
Shape of validation DataFrame: (33, 31)
Shape of test DataFrame: (22, 31)


In [21]:
feature_cols = [col for col in train_df.columns if col not in ['LAKE_NAME', 'Monit_MAP_CODE1', 'Split_Group'] + target_variables]

print(f"Number of feature columns: {len(feature_cols)}")
print("First 5 feature columns:")
print(feature_cols[:5])

Number of feature columns: 26
First 5 feature columns:
['', 'Year', 'Mean_EMV_Total', 'STD_EMV_Total', 'Mean_Sale_Value']


In [22]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for DecisionTreeRegressor
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

print("Imported DecisionTreeRegressor and GridSearchCV.")
print(f"Parameter grid for DecisionTreeRegressor: {param_grid}")

Imported DecisionTreeRegressor and GridSearchCV.
Parameter grid for DecisionTreeRegressor: {'max_depth': [3, 5, 7, 10], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}


In [23]:
best_cart_models = {}

for target in target_variables:
    print(f"\nPerforming GridSearchCV for target: {target}")

    # Prepare X_train and y_train
    X_train = train_df.select(feature_cols).to_pandas()
    y_train = train_df.select(target).to_pandas().squeeze()

    # Instantiate DecisionTreeRegressor
    dt = DecisionTreeRegressor(random_state=42)

    # Instantiate GridSearchCV
    grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)

    # Fit GridSearchCV
    grid_search.fit(X_train, y_train)

    # Store the best estimator
    best_cart_models[target] = grid_search.best_estimator_

    # Print results
    print(f"Best parameters for {target}: {grid_search.best_params_}")
    print(f"Best R-squared score for {target}: {grid_search.best_score_:.4f}")

print("\nGrid search for CART models completed.")



Performing GridSearchCV for target: Mean_Secchi_Depth_Result
Best parameters for Mean_Secchi_Depth_Result: {'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 10}
Best R-squared score for Mean_Secchi_Depth_Result: -0.8559

Performing GridSearchCV for target: Mean_Phosporus_Result
Best parameters for Mean_Phosporus_Result: {'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 10}
Best R-squared score for Mean_Phosporus_Result: -0.2085

Grid search for CART models completed.


In [24]:
from sklearn.ensemble import RandomForestRegressor

# Define the parameter grid for RandomForestRegressor
param_grid_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

print("Imported RandomForestRegressor.")
print(f"Parameter grid for RandomForestRegressor: {param_grid_rf}")

Imported RandomForestRegressor.
Parameter grid for RandomForestRegressor: {'n_estimators': [50, 100, 150], 'max_depth': [5, 10, 15], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}


In [25]:
best_rf_models = {}

for target in target_variables:
    print(f"\nPerforming GridSearchCV for Random Forest for target: {target}")

    # Prepare X_train and y_train
    X_train = train_df.select(feature_cols).to_pandas()
    y_train = train_df.select(target).to_pandas().squeeze()

    # Instantiate RandomForestRegressor
    rf = RandomForestRegressor(random_state=42)

    # Instantiate GridSearchCV
    grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, scoring='r2', n_jobs=-1)

    # Fit GridSearchCV
    grid_search_rf.fit(X_train, y_train)

    # Store the best estimator
    best_rf_models[target] = grid_search_rf.best_estimator_

    # Print results
    print(f"Best parameters for {target}: {grid_search_rf.best_params_}")
    print(f"Best R-squared score for {target}: {grid_search_rf.best_score_:.4f}")

print("\nGrid search for Random Forest models completed.")


Performing GridSearchCV for Random Forest for target: Mean_Secchi_Depth_Result
Best parameters for Mean_Secchi_Depth_Result: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
Best R-squared score for Mean_Secchi_Depth_Result: -0.5668

Performing GridSearchCV for Random Forest for target: Mean_Phosporus_Result
Best parameters for Mean_Phosporus_Result: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Best R-squared score for Mean_Phosporus_Result: -0.6480

Grid search for Random Forest models completed.


In [26]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

print("Imported evaluation metrics.")

Imported evaluation metrics.


In [27]:
model_performance = {}

# Prepare X_test once as features are the same for all targets
X_test = test_df.select(feature_cols).to_pandas()

for target in target_variables:
    print(f"\n--- Evaluating models for target: {target} ---")

    # Prepare y_test for the current target
    y_test = test_df.select(target).to_pandas().squeeze()

    # --- Evaluate CART Model ---
    cart_model = best_cart_models[target]
    cart_predictions = cart_model.predict(X_test)

    cart_r2 = r2_score(y_test, cart_predictions)
    cart_mae = mean_absolute_error(y_test, cart_predictions)
    cart_rmse = np.sqrt(mean_squared_error(y_test, cart_predictions))

    print(f"CART Model Performance for {target}:")
    print(f"  R-squared: {cart_r2:.4f}")
    print(f"  MAE: {cart_mae:.4f}")
    print(f"  RMSE: {cart_rmse:.4f}")

    model_performance[f'CART_{target}'] = {
        'R-squared': cart_r2,
        'MAE': cart_mae,
        'RMSE': cart_rmse
    }

    # --- Evaluate Random Forest Model ---
    rf_model = best_rf_models[target]
    rf_predictions = rf_model.predict(X_test)

    rf_r2 = r2_score(y_test, rf_predictions)
    rf_mae = mean_absolute_error(y_test, rf_predictions)
    rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))

    print(f"Random Forest Model Performance for {target}:")
    print(f"  R-squared: {rf_r2:.4f}")
    print(f"  MAE: {rf_mae:.4f}")
    print(f"  RMSE: {rf_rmse:.4f}")

    model_performance[f'RF_{target}'] = {
        'R-squared': rf_r2,
        'MAE': rf_mae,
        'RMSE': rf_rmse
    }

    # --- Compare Models ---
    if cart_r2 > rf_r2:
        print(f"Conclusion for {target}: CART model performed better (higher R-squared).")
    else:
        print(f"Conclusion for {target}: Random Forest model performed better (higher R-squared).")

print("\n--- Overall Model Performance Summary ---")
print(model_performance)



--- Evaluating models for target: Mean_Secchi_Depth_Result ---
CART Model Performance for Mean_Secchi_Depth_Result:
  R-squared: -58.8026
  MAE: 1.2843
  RMSE: 1.3376
Random Forest Model Performance for Mean_Secchi_Depth_Result:
  R-squared: -46.1389
  MAE: 1.1524
  RMSE: 1.1875
Conclusion for Mean_Secchi_Depth_Result: Random Forest model performed better (higher R-squared).

--- Evaluating models for target: Mean_Phosporus_Result ---
CART Model Performance for Mean_Phosporus_Result:
  R-squared: -2.9490
  MAE: 0.1584
  RMSE: 0.1893
Random Forest Model Performance for Mean_Phosporus_Result:
  R-squared: -2.7465
  MAE: 0.1617
  RMSE: 0.1843
Conclusion for Mean_Phosporus_Result: Random Forest model performed better (higher R-squared).

--- Overall Model Performance Summary ---
{'CART_Mean_Secchi_Depth_Result': {'R-squared': -58.802590720814166, 'MAE': 1.2843351919545098, 'RMSE': np.float64(1.337569785541465)}, 'RF_Mean_Secchi_Depth_Result': {'R-squared': -46.1388791902032, 'MAE': 1.1523

In [28]:
refitted_best_models = {}

# Combine train_df and test_df
combined_train_test_df = pl.concat([train_df, test_df])

for target in target_variables:
    print(f"\nRefitting best model for target: {target}")

    # Prepare X_combined and y_combined
    X_combined = combined_train_test_df.select(feature_cols).to_pandas()
    y_combined = combined_train_test_df.select(target).to_pandas().squeeze()

    # Retrieve the best model (Random Forest, as it performed better)
    best_model = best_rf_models[target]

    # Refit the model on the combined dataset
    best_model.fit(X_combined, y_combined)

    # Store the refitted model
    refitted_best_models[target] = best_model

    print(f"Model for {target} refitted successfully.")

print("\nRefitting of best models completed.")


Refitting best model for target: Mean_Secchi_Depth_Result
Model for Mean_Secchi_Depth_Result refitted successfully.

Refitting best model for target: Mean_Phosporus_Result
Model for Mean_Phosporus_Result refitted successfully.

Refitting of best models completed.


In [29]:
validation_performance = {}

# Prepare X_val once as features are the same for all targets
X_val = val_df.select(feature_cols).to_pandas()

for target in target_variables:
    print(f"\n--- Evaluating refitted model for target on Validation Set: {target} ---")

    # Prepare y_val for the current target
    y_val = val_df.select(target).to_pandas().squeeze()

    # Retrieve the refitted model
    refitted_model = refitted_best_models[target]

    # Make predictions on X_val
    val_predictions = refitted_model.predict(X_val)

    # Calculate evaluation metrics
    val_r2 = r2_score(y_val, val_predictions)
    val_mae = mean_absolute_error(y_val, val_predictions)
    val_rmse = np.sqrt(mean_squared_error(y_val, val_predictions))

    # Print results
    print(f"Refitted Model Performance on Validation Set for {target}:")
    print(f"  R-squared: {val_r2:.4f}")
    print(f"  MAE: {val_mae:.4f}")
    print(f"  RMSE: {val_rmse:.4f}")

    # Store metrics
    validation_performance[f'Refitted_RF_{target}'] = {
        'R-squared': val_r2,
        'MAE': val_mae,
        'RMSE': val_rmse
    }

print("\n--- Overall Refitted Model Performance on Validation Set ---")
print(validation_performance)


--- Evaluating refitted model for target on Validation Set: Mean_Secchi_Depth_Result ---
Refitted Model Performance on Validation Set for Mean_Secchi_Depth_Result:
  R-squared: 0.0437
  MAE: 0.7147
  RMSE: 0.8284

--- Evaluating refitted model for target on Validation Set: Mean_Phosporus_Result ---
Refitted Model Performance on Validation Set for Mean_Phosporus_Result:
  R-squared: -2.3472
  MAE: 0.0706
  RMSE: 0.0740

--- Overall Refitted Model Performance on Validation Set ---
{'Refitted_RF_Mean_Secchi_Depth_Result': {'R-squared': 0.04366748426061484, 'MAE': 0.7146721920909161, 'RMSE': np.float64(0.828413304201175)}, 'Refitted_RF_Mean_Phosporus_Result': {'R-squared': -2.347249729895068, 'MAE': 0.07060640639692127, 'RMSE': np.float64(0.07396098370408526)}}


## Final Task

Summarize the entire process and what we learned

## Overall Summary

In this project we tried to predict two water quality indicators, Mean Secchi Depth and Mean Phosphorus, using tree-based models. The dataset was split into training, validation, and test groups, but the models struggled to make accurate predictions. On the training data, both CART and Random Forest models had negative R-squared values, meaning they performed worse than just guessing the average. When tested, Random Forest did slightly better than CART, but still had very poor results with large negative R-squared scores. Even after refitting the best Random Forest models, performance on the validation set was weak, with one target showing only a very small positive R-squared and the other still negative. Overall, this shows that the current features don't explain the water quality measures well, and the models can't capture the relationships in the data. To improve, we'll need stronger feature engineering, maybe adding time-series information, and trying more advanced algorithms like XGBoost or LightGBM that might handle complex patterns better.
